In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install evaluate
!pip install jiwer
!pip install gradio
!pip install  bitsandbytes datasets accelerate
!pip install  peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 42.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 102.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.5 MB/s eta 0:00:00


In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2").to(device)



preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset, Audio

dataset = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test[:5]",trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))


n_shards.json:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

mr_train_0.tar:   0%|          | 0.00/82.3M [00:00<?, ?B/s]

mr_dev_0.tar:   0%|          | 0.00/65.5M [00:00<?, ?B/s]

mr_test_0.tar:   0%|          | 0.00/69.7M [00:00<?, ?B/s]

mr_other_0.tar:   0%|          | 0.00/110M [00:00<?, ?B/s]

...


Reading metadata...: 1816it [00:00, 83776.29it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2819it [00:00, 88989.31it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 2237it [00:00, 10449.16it/s]A


In [5]:
import evaluate

wer_metric = evaluate.load("wer")

def compute_wer(batch):
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="mr", task="transcribe")
    predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return {"transcription": transcription}

results = dataset.map(compute_wer)
wer = wer_metric.compute(predictions=results["transcription"], references=results["sentence"])
print(f"Word Error Rate (WER): {wer}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Word Error Rate (WER): 1.0606060606060606


In [7]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from datasets import load_dataset, Audio
import evaluate

# Load model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2").to(device)

# Load dataset
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test[:5]")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Load WER metric
wer_metric = evaluate.load("wer")

# Define function to compute WER and print transcriptions
def compute_wer(batch):
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="mr", task="transcribe")
    predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return {"transcription": transcription, "ground_truth": batch["sentence"]}

# Map dataset to get results
results = dataset.map(compute_wer)

# Print transcriptions and ground truths
for i in range(len(results)):
    print(f"Ground Truth: {results[i]['ground_truth']}")
    print(f"Transcription: {results[i]['transcription']}\n")

# Calculate and print WER
wer = wer_metric.compute(predictions=results["transcription"], references=results["ground_truth"])
print(f"Word Error Rate (WER): {wer}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Ground Truth: स्वेच्छानिवृतीचा पध्दत मोठ्या प्रमाणात अंमलात आल्यापासून या दुखण्याने अनेक घरांत प्रवेश केला आहे.
Transcription:  स्विच्चे नरूत्तिची पदल मोठा प्रमणाद अमलाद अनले बजोंद या दुपने ने अने कराट प्रविश केला है

Ground Truth: गोविंदपंत व मीराबाई हे त्यांचे आजोबा आजी होत.
Transcription:  गोविंड पंकावा मीरा बाई हे कैन से आजबा आजी होत।

Ground Truth: रावणाला संगीताची आवड होती.
Transcription:  रावनाला संगेताचे आवड होते

Ground Truth: चारुता सागर दिनकर दत्तात्रेय भोसले
Transcription:  चारूर्ता सागतिनकर्ड़्टात्रेव्भूस्लिप्

Ground Truth: खमंग वास येतोय.
Transcription:  पर में गए वाई सीखे

Word Error Rate (WER): 1.0606060606060606


In [14]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from datasets import load_dataset, Audio
import evaluate

# Load model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2").to(device)

# Load dataset
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "mr", split="test")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Load WER metric
wer_metric = evaluate.load("wer")

# Define function to compute WER and print transcriptions
def compute_wer(batch):
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="mr", task="transcribe")
    predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
    
    # Decode the output without skipping special tokens (so no normalization is applied)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)[0]
    
    return {"transcription": transcription, "ground_truth": batch["sentence"]}

# Map dataset to get results
results = dataset.map(compute_wer)

# Print transcriptions and ground truths
for i in range(len(results)):
    print(f"Ground Truth: {results[i]['ground_truth']}")
    print(f"Transcription: {results[i]['transcription']}\n")

# Calculate and print WER
wer = wer_metric.compute(predictions=results["transcription"], references=results["ground_truth"])
print(f"Word Error Rate (WER): {wer}")


Map:   0%|          | 0/1816 [00:00<?, ? examples/s]

Ground Truth: स्वेच्छानिवृतीचा पध्दत मोठ्या प्रमाणात अंमलात आल्यापासून या दुखण्याने अनेक घरांत प्रवेश केला आहे.
Transcription: <|startoftranscript|><|mr|><|transcribe|><|notimestamps|> स्विच्चे नरूत्तिची पदल मोठा प्रमणाद अमलाद अनले बजोंद या दुपने ने अने कराट प्रविश केला है

Ground Truth: गोविंदपंत व मीराबाई हे त्यांचे आजोबा आजी होत.
Transcription: <|startoftranscript|><|mr|><|transcribe|><|notimestamps|> गोविंड पंकावा मीरा बाई हे कैन से आजबा आजी होत।

Ground Truth: रावणाला संगीताची आवड होती.
Transcription: <|startoftranscript|><|mr|><|transcribe|><|notimestamps|> रावनाला संगेताचे आवड होते

Ground Truth: चारुता सागर दिनकर दत्तात्रेय भोसले
Transcription: <|startoftranscript|><|mr|><|transcribe|><|notimestamps|> चारूर्ता सागतिनकर्ड़्टात्रेव्भूस्लिप्

Ground Truth: खमंग वास येतोय.
Transcription: <|startoftranscript|><|mr|><|transcribe|><|notimestamps|> पर में गए वाई सीखे

Ground Truth: प्रवेशद्वाराच्या बाजूला एका कोनाड्याच्या भिंतीवर दोन ओळींमध्ये लिहिलेला ब्राम्ही लिपीतील शिलालेख आहे.
Tra

In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from datasets import load_dataset, Audio
import evaluate

# Load model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2").to(device)

# Load dataset
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "as", split="test")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Load WER metric
wer_metric = evaluate.load("wer")

# Define function to compute WER and print transcriptions
def compute_wer(batch):
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="as", task="transcribe")
    predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
    
    # Decode the output without skipping special tokens (so no normalization is applied)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)[0]
    
    return {"transcription": transcription, "ground_truth": batch["sentence"]}

# Map dataset to get results
results = dataset.map(compute_wer)

# Print transcriptions and ground truths
for i in range(len(results)):
    print(f"Ground Truth: {results[i]['ground_truth']}")
    print(f"Transcription: {results[i]['transcription']}\n")

# Calculate and print WER
wer = wer_metric.compute(predictions=results["transcription"], references=results["ground_truth"])
print(f"Word Error Rate (WER): {wer}")


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 163it [00:00, 93359.49it/s]


Map:   0%|          | 0/308 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Ground Truth: দঢ়াই দঢ়াই মাতি গৈছিল সকলোকে।
Transcription: <|startoftranscript|><|as|><|transcribe|><|notimestamps|> Dohai dohai mati goisari hoko luke

Ground Truth: আমাৰ ঘৰত তলা মৰা অৱস্থাত ওছৰৰে ঘৰত মানুহ নথকা সময়ত চুৰি হৈছিল।
Transcription: <|startoftranscript|><|as|><|transcribe|><|notimestamps|> अमार खरत तला मरा अबुस्थाट, उसररे खरत मानुद, नथका हमारा छुरी होजिल।

Ground Truth: মন ভালো লগা নাই, বেয়াও, নগা নাই।
Transcription: <|startoftranscript|><|as|><|transcribe|><|notimestamps|> मौन भालू लोगा नाई, बेआउ लोगा नाई।

Ground Truth: ইয়াৰ পাচত সূৰ্য্যই যেতিয়া ৰোগীক কেনে দেখিলে সুধিলে ডাক্তৰে ক'ব লগাত পৰিল।
Transcription: <|startoftranscript|><|as|><|transcribe|><|notimestamps|> इयार पसट हुछोई जेतियार रोगी केने देखिले हुडिले, दाक्तरे कोगलो गात परिल

Ground Truth: তেওঁ ৰামৰ এজন ভাল অনুগামী।
Transcription: <|startoftranscript|><|as|><|transcribe|><|notimestamps|> एओ रामोर अजन भाल उनुगमी

Ground Truth: যদিও এই উদ্যোগৰ ইতিহাস পুৰণি।
Transcription: <|startoftranscript|><|as|><|transcribe|><|n